## Network implementation

In [ ]:
from datetime import datetime
import gzip, pickle
import numpy as np
import random
import matplotlib.pyplot as plt

f = gzip.open('mnist.pkl.gz')
data = pickle.load(f, encoding='latin1')

train_data = data[0][0]
valid_data, valid_labels = data[1]
test_data, test_labels = data[2]

# onehot encoding for valid and test labels
train_labels = np.zeros((train_data.shape[0], 10))
train_labels[np.arange(train_labels.shape[0]), data[0][1]] = 1

In [6]:
class NN(object):
    
    def __init__(self, hidden_dims=(1024,2048), n_hidden=2, mode='train', datapath=None, model_path=None,
                batchsize=64, lr=0.0001, delta=.5, activation="ReLU", initialization="normal", epsilon=1e-9):
        
        self.dims = (784,) + hidden_dims + (10,)
        self.n_hidden = n_hidden
        self.mode = mode
        self.datapath = datapath
        self.model_path = model_path
        self.batchsize = batchsize
        self.lr = lr
        self.delta = delta
        self.epsilon = epsilon
        
        # set activation function for hidden layers
        if activation == "ReLU":
            self.activation = self.ReLU
            self.activation_prime = self.ReLU_prime
        elif activation == "sigmoid":
            self.activation = self.sigmoid
            self.activation_prime = self.sigmoid_prime
        elif activation == "tanh":
            self.activation = self.tanh
            self.activation_prime = self.tanh_prime
        else:
            raise ValueError('Invalid activation function specified: ' + str(activation))
        
        # network parameters
        self.W = []
        self.b = [np.zeros(self.dims[i]) for i in range(1, len(self.dims))]
        
        # weight initialization
        self.initialize_weights(n_hidden, self.dims, initialization)
        
        # keep count of completed training epochs
        self.epoch_cnt = 1
    
    ############################
    #  Weight initializations  #
    ############################
    
    def initialize_weights(self, n_hidden, dims, option):
        
        if option == "zero":
            for i in range(n_hidden + 1):
                self.W.append( np.zeros((dims[i+1], dims[i])) )
        
        elif (option == "normal"):
            for i in range(n_hidden + 1):
                self.W.append( np.random.normal(0, 1, (dims[i+1], dims[i])) )
        
        elif (option == "glorot"):
            for i in range(n_hidden + 1):
                di = np.sqrt(6/(dims[i]+dims[i+1]))
                self.W.append( np.random.uniform(-di, di, (dims[i+1], dims[i])) )
        
        else:
            raise ValueError('Invalid weight initialization specified: ' + str(option))
    
    #################################################
    #  Activation functions  and their derivatives  #
    #################################################
    
    def ReLU(self, input):
        return np.maximum(0, input)
    
    def ReLU_prime(self, input):
        return np.where(input > 0, 1, 0)
    
    def sigmoid(self, input):
        return 1 / (1 + np.exp(-input))
    
    def sigmoid_prime(self, input):
        return self.sigmoid(input) * (1 - self.sigmoid(input))
    
    def tanh(self, input):
        return 2 / (1 + np.exp(-2*input)) - 1
    
    def tanh_prime(self, input):
        return 1 - self.tanh(input)**2
    
    def softmax(self, input):
        rescaled = input - np.amax(input, axis=1)[:,np.newaxis] # for numerical stability
        input_exp = np.exp(rescaled)
        return input_exp / np.sum(input_exp, axis=1)[:,np.newaxis]
    
    ########################################
    #  Forward, backward, loss and update  #
    ########################################
    
    def forward(self, input):
        
        # cache stores z = Wx + b of every layer for backprop (z=x for first layer)
        cache = [input]
        
        out = input
        for i in range(self.n_hidden):
            out = out @ self.W[i].T + self.b[i] # compute z = Wx + b
            cache.append(out) # store z for backprop
            out = self.activation(out) # compute a = activation(z)
        
        out = out @ self.W[-1].T + self.b[-1]
        cache.append(out)
        out = self.softmax(out)
        
        return out, cache
    
    def loss(self, prediction, labels):
        return -np.sum(labels * np.log(prediction + self.epsilon)) / self.batchsize
    
    def backward(self, cache, labels):
        
        # cache contains z = Wx + b of every layer (z=x for first layer)
        
        grads_a = [0] * (self.n_hidden + 1)
        
        grads_a[-1] = self.softmax(cache[-1]) - labels
        for i in range(self.n_hidden-1, -1, -1):
            grads_a[i] = grads_a[i+1] @ self.W[i+1] * self.activation_prime(cache[i+1])
        
        grads_W = [0] * (self.n_hidden + 1)
        grads_b = [0] * (self.n_hidden + 1)
        
        for i in range(self.n_hidden, -1, -1):
            grads_W[i] = grads_a[i].T @ self.activation(cache[i]) / self.batchsize
            grads_b[i] = np.mean(grads_a[i], axis=0)
            
        return (grads_W, grads_b)
    
    
    def update(self, grads, lr):
        for i in range(self.n_hidden + 1):
            self.W[i] -= lr * grads[0][i]
            self.b[i] -= lr * grads[1][i]
        
    ##############################
    #  Train and test functions  #
    ##############################
    
    def train(self, train_data, train_labels, n_epochs):
        
        for epoch in range(n_epochs):
            
            print(datetime.now(), "-", "Epoch", self.epoch_cnt, end=": ") 
            
            # learning rate for this epoch (doesn't change during first 5 epochs)
            t = max(1, self.epoch_cnt-5)
            lr = self.lr / t**self.delta
            
            epoch_loss = 0
            
            for i in range(train_data.shape[0] // self.batchsize):
            
                # start and end of mini-batch
                start = i * self.batchsize
                end = (i+1) * self.batchsize
                
                # forward pass and epoch loss
                predictions, cache = self.forward(train_data[start:end])
                epoch_loss += self.loss(predictions, train_labels[start:end])
                
                # backward pass and update
                grads = self.backward(cache, train_labels[start:end])
                self.update(grads, lr)
            
            print("loss =", epoch_loss)
            self.epoch_cnt += 1
        
        # self.finite_diff()
        
        
    def test(self, test_data, test_labels):
        
        predictions, cache = self.forward(test_data)
        successes = (np.argmax(predictions, axis=1) == test_labels)
        
        print(str(100 * np.sum(successes) / test_data.shape[0]) + "% success")
    
    ############################################
    #  Finite difference computation funciton  #
    ############################################
    
    def finite_diff(self, sample, label):
        random.seed(9001)
        #first layer weight
        weight_save = np.copy(self.W[1])
        
        #choose five random value from set N
        num_N = 5
        P = np.min([10, hidden_dims[1]])
        w_size = 256
        N = []
        K = [1, 5]
        fi_grad = np.zeros((num_N, P, w_size))
        for num_N in range(num_N):
            i = random.randint(0,5)
            k = K[random.randint(0,1)]
            N.append(k * (10**i))
        
        N.sort()
        print(N)
        #calculate finite difference for first 10 weights in first layer
        for i in range(len(N)):
            epsilon = 1 / N[i]
            print("Set Epsilon = ", epsilon)
            grad = np.zeros((P,w_size))
            for p in range(P):
                #restore the original weight before going to next finite difference calculation
                self.W[1] = np.copy(weight_save)
                for z in range(w_size):
                    w1 = np.copy(self.W[1])
                    w2 = np.copy(self.W[1])

                    w1[p][z] -= epsilon
                    w2[p][z] += epsilon

                    #theta(i) - epsilon
                    self.W[1] = np.copy(w1)
                    L1, dum = self.forward(sample)
                    L1_loss = self.loss(L1, label)
                    #theta(i) + epsilon
                    self.W[1] = np.copy(w2)
                    L2, dum = self.forward(sample)
                    L2_loss = self.loss(L2, label)
                    g = (L2_loss-L1_loss)/(2*epsilon)
                    grad[p,z] = g
                    
                    
            fi_grad[i,:] = grad
            
        self.W[1] = np.copy(weight_save)
        #True gradient Calculation
        prediction, cache = self.forward(sample)
        loss = self.loss(prediction, label)
        grads = self.backward(cache, label)
        
        #fi_grad(finite_difference)
        
        return N, fi_grad, grads[0][1][0:10,:]

## Network parameters

In [7]:
hidden_dims = (256, 2048)
n_hidden = 2
mode = 'train'
datapath = None
model_path = None
batchsize = 32
lr = .01
delta = .5
activation = "sigmoid"

## Trying with glorot initialization

In [8]:
np.random.seed(6135)
net_glorot = NN(hidden_dims, n_hidden, mode, datapath, model_path,
                batchsize, lr, delta, activation, initialization="glorot")

In [9]:
for i in range(25):
    net_glorot.train(train_data, train_labels, 1)
    net_glorot.test(valid_data, valid_labels)

2019-02-11 13:45:25.914358 - Epoch 1: loss = 3569.9555987238964
30.32% success
2019-02-11 13:45:39.873295 - Epoch 2: loss = 3263.523134930674
57.21% success
2019-02-11 13:45:53.370384 - Epoch 3: loss = 2664.6264730668645
64.98% success
2019-02-11 13:46:07.169520 - Epoch 4: loss = 1956.3737065367163
70.13% success
2019-02-11 13:46:20.742740 - Epoch 5: loss = 1485.194446435469
76.76% success
2019-02-11 13:46:34.283123 - Epoch 6: loss = 1213.7158886150455
80.49% success
2019-02-11 13:46:47.950787 - Epoch 7: loss = 1058.685302479545
83.61% success
2019-02-11 13:47:01.472951 - Epoch 8: loss = 977.520159704424
84.57% success
2019-02-11 13:47:15.177574 - Epoch 9: loss = 924.0104470025773
85.41% success
2019-02-11 13:47:28.757628 - Epoch 10: loss = 884.8193694388555
85.86% success
2019-02-11 13:47:42.248953 - Epoch 11: loss = 854.2567317080496
86.17% success
2019-02-11 13:47:55.746655 - Epoch 12: loss = 829.4035974569988
86.63% success
2019-02-11 13:48:09.281695 - Epoch 13: loss = 808.58277501

In [10]:
net_glorot.test(train_data, data[0][1])
# net_glorot.test(test_data, test_labels)

87.74% success


## Trying with normal initialization

In [11]:
np.random.seed(6135)
net_normal = NN(hidden_dims, n_hidden, mode, datapath, model_path,
                batchsize, lr, delta, activation, initialization="normal")

In [ ]:
for i in range(25):
    net_normal.train(train_data, train_labels, 1)
    net_normal.test(valid_data, valid_labels)

2019-02-11 13:51:08.050338 - Epoch 1: loss = 7865.612901551834
74.33% success
2019-02-11 13:51:21.212898 - Epoch 2: loss = 3549.3301700791767
79.96% success
2019-02-11 13:51:34.879429 - Epoch 3: loss = 2706.8724582193113
81.96% success
2019-02-11 13:51:48.531626 - Epoch 4: loss = 2263.4461377864745
83.43% success
2019-02-11 13:52:02.244016 - Epoch 5: loss = 1969.6895710217789
84.15% success
2019-02-11 13:52:16.043214 - Epoch 6: loss = 1751.5099465231397
84.69% success
2019-02-11 13:52:29.816065 - Epoch 7: loss = 1572.9959022005712
85.14% success
2019-02-11 13:52:43.686251 - Epoch 8: loss = 1465.0835358445177
85.37% success
2019-02-11 13:52:57.384956 - Epoch 9: loss = 1387.6496580577525
85.65% success
2019-02-11 13:53:11.196962 - Epoch 10: loss = 1327.146343968756
85.93% success
2019-02-11 13:53:26.151738 - Epoch 11: 

In [ ]:
net_normal.test(train_data, data[0][1])
# net_normal.test(test_data, test_labels)

## Trying with zero initialization

In [ ]:
np.random.seed(6135)
net_zero = NN(hidden_dims, n_hidden, mode, datapath, model_path,
              batchsize, lr, delta, activation, initialization="zero")

In [ ]:
for i in range(25):
    net_zero.train(train_data, train_labels, 1)
    net_zero.test(valid_data, valid_labels)

In [ ]:
net_zero.test(train_data, data[0][1])
# net_zero.test(test_data, test_labels)

In [ ]:
#finite difference gradient for first 10 weights in first layer
NN, f_grad, t_grad = net_glorot.finite_diff(np.expand_dims(train_data[0],axis=0), np.expand_dims(train_labels[0],axis=0))
print (NN)

#calculate maximum difference
fgd = []
for i in range(len(NN)):
    euclidean = np.sum((t_grad-f_grad[i])**2, axis=1)
    fgd.append(np.max(euclidean))

#plot the maximum difference
plt.figure()
plt.plot(NN, np.log(fgd))
plt.xlabel('N')
plt.ylabel('Log of maximum difference')
plt.grid(True)
plt.show()